In [74]:
import tractography_latest as tract
reload(tract)

<module 'tractography_latest' from 'tractography_latest.py'>

In [2]:
test = tract.tiff_stack_to_array('/home/tractography_test/CTT/demo/data/')

(90, 90, 90)


In [41]:
from scipy import io

In [43]:
stackmat = io.loadmat('stack_image.mat')

In [51]:
np_stackmat = stackmat['stack']

In [52]:
print np_stackmat.shape

(90, 90, 90)


In [53]:
dogsigmaArr = [1]
gausigmaArr = [0.5]
img_data = np_stackmat

In [55]:
import os
import numpy as np
import math
import SimpleITK as sitk
from scipy import ndimage
import nibabel as nib
from PIL import Image
import scipy.misc
from scipy import signal

In [151]:
# Helper function similar to MATLAB
def cropvol(vol, sz):
    print np.size(vol);
    vol[0:sz, :, :, :] = 0;
    vol[-1-sz+1:-1, :, :, :] = 0;

    vol[:, 0:sz, :, :] = 0;
    vol[:, -1-sz+1:-1, :, :] = 0;

    vol[:, :, 0:sz, :] = 0;
    vol[:, :, -1-sz+1:-1, :] = 0;
    return vol;

for ii in range(len(dogsigmaArr)):
    dogsigma = dogsigmaArr[ii];
    print "Start DoG Sigma on " + str(dogsigma);

    # Generate dog kernels
    dogkercc = tract.doggen([dogsigma, dogsigma, dogsigma]);
    dogkercc = np.transpose(dogkercc, (0, 2, 1));  # annoying

    #print dogkercc.shape;
    #print dogkercc[:, :, 0];

    dogkerrr = np.transpose(dogkercc, (1, 0, 2));

    #print dogkerrr[:, :, 0];
    dogkerzz = np.transpose(dogkercc, (0, 2, 1));

    #print dogkerzz[:, :, 0];

    for jj in range(len(gausigmaArr)):
        gausigma = gausigmaArr[jj];

        print "Start Gauss Sigma with gausigma = " + str(gausigma);

        print "Generating Gaussian kernel..."
        gaussker = np.single(tract.gaussgen([gausigma, gausigma, gausigma]));

        # Generate half size kernel
        halfsz = (max(len(dogkercc), len(gaussker)) + 1) / 2;

        # Compute gradients
        grr = signal.convolve((img_data).astype('float64'), dogkerrr.astype('float64'), 'same');
        grr.astype('float64');

        #print grr[:, :, 0];

        gcc = signal.convolve(img_data.astype('float64'), dogkercc.astype('float64'), 'same');
        gcc.astype('float64');
        
        #print gcc[:, :, 0];

        gzz = signal.convolve(img_data.astype('float64'), dogkerzz.astype('float64'), 'same');
        gzz.astype('float64');
        
        #print gzz[:, :, 0];

        # Compute gradient products
        gprrrr = np.multiply(grr.astype('float64'), grr.astype('float64'));

        #print gprrrr[:, :, 0];

        gprrcc = np.multiply(grr.astype('float64'), gcc.astype('float64'));

        #print gprrcc[:, :, 0];

        gprrzz = np.multiply(grr.astype('float64'), gzz.astype('float64'));

        #print gprrzz[:, :, 0]

        gpcccc = np.multiply(gcc.astype('float64'), gcc.astype('float64'));
        gpcczz = np.multiply(gcc.astype('float64'), gzz.astype('float64'));
        gpzzzz = np.multiply(gzz.astype('float64'), gzz.astype('float64'));

        # Compute gradient amplitudes
        # print ga.dtype;
        ga = np.sqrt(gprrrr.astype('float64') + gpcccc.astype('float64') + gpzzzz.astype('float64'));

        #print ga[:, :, 0];

        #print "GA SHAPE:"
        #print ga.shape;

        # Convert numpy ndarray object to Nifti data type
        gradient_amplitudes_data = nib.Nifti1Image(ga, affine=np.eye(4));

        # Save gradient amplitudes image 
        nib.save(gradient_amplitudes_data, 'gradient_amplitudes.nii.gz');

        # Compute gradient vectors
        gv = np.concatenate((grr[..., np.newaxis], gcc[..., np.newaxis], gzz[..., np.newaxis]), axis = 3);

        #print gv[:, :, 0, 0];

        gv = np.divide(gv, np.tile(ga[..., None], [1, 1, 1, 3]));
        #print gv[:, :, 0, 1];

        #print "GV SHAPE:"
        #print gv.shape;

        # Convert numpy ndarray object to Nifti data type
        gradient_vectors_data = nib.Nifti1Image(gv, affine=np.eye(4));

        # Save gradient vectors
        nib.save(gradient_vectors_data, 'gradient_vectors.nii.gz');

        #print gaussker[:, :, 0];

        print "Blurring gradient products..."
        gprrrrgauss = signal.convolve(gprrrr, gaussker, "same");

        #print gprrrrgauss[:, :, 0];

        gprrccgauss = signal.convolve(gprrcc, gaussker, "same");

        #print gprrccgauss[:, :, 0];

        gprrzzgauss = signal.convolve(gprrzz, gaussker, "same");
        gpccccgauss = signal.convolve(gpcccc, gaussker, "same");
        gpcczzgauss = signal.convolve(gpcczz, gaussker, "same");
        gpzzzzgauss = signal.convolve(gpzzzz, gaussker, "same");

Start DoG Sigma on 1
Start Gauss Sigma with gausigma = 0.5
Generating Gaussian kernel...
Blurring gradient products...


In [152]:
print gzz[0:9, 0:9, 0]

[[ 1485.11507547  2071.73673277  2242.19514136  2231.21226528
   2169.71230573  2131.14860771  2144.21369722  2176.68696989
   2210.57279593]
 [ 1985.86695562  2736.41284045  2949.17663432  2952.51391776
   2891.66865107  2850.90696594  2872.23214105  2909.58229688  2960.2761203 ]
 [ 2060.58107332  2814.12897507  3039.44802461  3077.26715829
   3053.30529088  3045.90199304  3100.81634788  3150.5784548   3177.63040656]
 [ 2073.56622628  2829.78147097  3056.06891825  3097.43817689
   3091.36152156  3109.54591955  3190.40000762  3252.08834801
   3251.24517476]
 [ 2130.08080015  2927.05164612  3159.52013257  3188.47020313
   3185.27362037  3208.5941074   3267.64983135  3298.6687641   3275.93390289]
 [ 2228.88831947  3067.1149305   3298.35090291  3307.9474194   3299.17071263
   3324.33459841  3357.36839937  3350.95429832  3311.70735126]
 [ 2308.37204562  3163.01676715  3379.63832411  3361.108583    3335.6039943
   3356.3865674   3384.83489744  3371.01617981  3325.35337043]
 [ 2316.29184544 

In [153]:
print gprrrr[0:9, 0:9, 0]

[[  2.15173330e+06   4.04055725e+06   4.56351136e+06   4.43245543e+06
    4.19856142e+06   4.10161860e+06   4.18192796e+06   4.28695069e+06
    4.48564628e+06]
 [  1.54011483e+05   2.22280424e+05   2.43727849e+05   2.98579537e+05
    3.70143601e+05   4.49064983e+05   5.45307790e+05   6.17411084e+05
    6.55960296e+05]
 [  4.53097026e+02   7.34753599e+03   7.30973297e+03   1.10006665e-01
    1.12777496e+04   3.76819195e+04   7.81682866e+04   1.16215954e+05
    9.65906621e+04]
 [  1.53561153e+04   2.59143472e+04   2.08638053e+04   2.16596706e+04
    2.92778878e+04   2.52637317e+04   1.26041648e+04   7.89938422e+03
    4.10927683e+03]
 [  4.57328284e+04   9.17728045e+04   8.61366133e+04   6.97908433e+04
    6.35182012e+04   4.57404261e+04   1.14707138e+04   6.08465697e+01
    1.00733437e+03]
 [  3.19573526e+04   5.98547181e+04   5.99779215e+04   4.10014865e+04
    3.01603224e+04   2.66879858e+04   1.61957932e+04   4.65414661e+03
    4.67616188e+02]
 [  1.39087445e+03   4.19281251e+03   8.

In [154]:
print grr[0:9, 0:9, 0]

[[  1.46687876e+03   2.01011374e+03   2.13623766e+03   2.10533974e+03
    2.04903915e+03   2.02524532e+03   2.04497627e+03   2.07049528e+03
    2.11793444e+03]
 [  3.92442967e+02   4.71466249e+02   4.93688007e+02   5.46424319e+02
    6.08394281e+02   6.70123110e+02   7.38449585e+02   7.85755104e+02
    8.09913759e+02]
 [ -2.12860759e+01  -8.57177694e+01  -8.54969764e+01  -3.31672527e-01
    1.06196749e+02   1.94118313e+02   2.79585920e+02   3.40904611e+02
    3.10790383e+02]
 [  1.23919794e+02   1.60979338e+02   1.44443087e+02   1.47172248e+02
    1.71107825e+02   1.58945688e+02   1.12268272e+02   8.88784801e+01
    6.41036413e+01]
 [  2.13852352e+02   3.02940266e+02   2.93490397e+02   2.64179566e+02
    2.52028175e+02   2.13870115e+02   1.07101418e+02   7.80042113e+00
   -3.17385313e+01]
 [  1.78766195e+02   2.44652239e+02   2.44903903e+02   2.02488238e+02
    1.73667275e+02   1.63364579e+02   1.27262694e+02   6.82213061e+01
    2.16244350e+01]
 [  3.72944292e+01   6.47519306e+01   9.

In [155]:
print (grr[0, 0, 0]).astype('uint64') * (grr[0, 0, 0]).astype('uint64')

2149156


In [156]:
print ga[0:9, 0:9, 0]

[[ 2612.8196015   2961.13518089  3097.85110806  3069.48095848
   2985.86917415  2939.97140158  2963.38837618  3004.22229509
   3061.47940513]
 [ 2875.77077483  2892.05615444  2991.61191337  3004.60364415
   2956.49730676  2928.72630394  2966.87474108  3014.30262477
   3070.07025019]
 [ 2903.43193596  2921.34689306  3044.13612184  3077.27593834
   3055.15187521  3054.20041696  3117.9468136   3170.01511581
   3193.04530368]
 [ 2914.76618761  2932.52561153  3063.73643416  3101.48782674
   3096.60349278  3116.78143871  3198.63046888  3254.59146452
   3252.01568672]
 [ 3039.38264649  3043.08625672  3175.83830797  3199.88178341
   3195.60999055  3216.46877697  3271.00008386  3299.07775982
   3276.74735535]
 [ 3190.71121849  3180.95338043  3308.90999297  3314.23261642
   3303.97161514  3328.44249674  3359.79400547  3351.68194005
   3313.18737057]
 [ 3284.51240448  3274.12328293  3381.97318573  3361.92306859
   3335.81469889  3356.465608    3384.89487159  3371.68017811
   3328.47009419]
 [ 328

In [157]:
reload(tract)
fsl, dtk = tract.generate_FSL_and_DTK_structure_tensor(np_stackmat, 'test_data', dogsigmaArr=[1], gausigmaArr=[0.5]);

Start DoG Sigma on 1
Start Gauss Sigma with gausigma = 0.5
Generating Gaussian kernel...
Blurring gradient products...
Saving a copy for this Gaussian sigma...
4374000
4374000
Completed computing structure tensor on test_data!


In [9]:
mask = tract.tiff_stack_to_nii('/home/tractography_test/CTT/demo/mask-brain/', 'brainmask')

In [120]:
import nibabel as nib
MATLAB_output = nib.load("/home/tractography_test/CTT/demo/result/dog1gau0.5/dtk_tensor.nii.gz")

In [121]:
MATLAB_np_array = MATLAB_output.get_data()

In [122]:
print dtk.shape

(90, 90, 90, 6)


In [158]:
dtk[:, :, :, 0]

array([[[      0.        ,       0.        ,       0.        , ...,
               0.        ,       0.        ,       0.        ],
        [      0.        ,       0.        ,       0.        , ...,
               0.        ,       0.        ,       0.        ],
        [      0.        ,       0.        ,       0.        , ...,
               0.        ,       0.        ,       0.        ],
        ..., 
        [      0.        ,       0.        ,       0.        , ...,
               0.        ,       0.        ,       0.        ],
        [      0.        ,       0.        ,       0.        , ...,
               0.        ,       0.        ,       0.        ],
        [      0.        ,       0.        ,       0.        , ...,
               0.        ,       0.        ,       0.        ]],

       [[      0.        ,       0.        ,       0.        , ...,
               0.        ,       0.        ,       0.        ],
        [      0.        ,       0.        ,       0.       

In [130]:
MATLAB_np_array[:, :, :, 0]

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0., 

In [169]:
import numpy as np
truth_boolean = np.isclose(dtk, MATLAB_np_array, rtol = 1e-4)

In [170]:
print MATLAB_np_array.shape

(90, 90, 90, 6)


In [171]:
correct_number = np.sum(truth_boolean == True);  # Total possible = 4,374,000
print correct_number

4238655


In [173]:
## Percentage correct (at the 1e-4 value)
print correct_number / (4374000.0)

0.969056927298
